# Classification des données en utilisant le SVM

## Importation des données

In [ ]:
import pymongo
import sklearn as sk
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.decomposition import PCA
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

On prendre 3 variables pour le SVM car les donnés sont issus 

In [ ]:
tmp= pd.read_csv("../../data/export_data_labeled.csv",header=None)
df=tmp.head(20000)
X1=df.drop(df.columns[-1], axis=1)
X=X1.drop(df.columns[[3,4,5,6,7,8,9]], axis=1).values
labels=df.iloc[:, -1].replace([1.0,2.0,4.0], 0)
labels=labels.replace(3.0, 1)
Y=labels.values
print(labels.value_counts())
print(X)


## Séparation des données labélisées en Apprentisage , Test et Validation

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

## Classification avec SVM

### Déterminaison de la meilleure combinaison d'hyperparamètres

Définition d'une fonction pour afficher une matrice de confusion

In [ ]:
def display_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    
    # Création de la figure
    fig, ax = plt.subplots()

    # Création de la heatmap
    heatmap = ax.imshow(cm, cmap='Blues')

    # Ajout des valeurs dans les cellules de la heatmap
    for i in range(len(cm)):
        for j in range(len(cm[i])):
            ax.text(j, i, cm[i][j], ha='center', va='center', color='black')

    # Définition des étiquettes des axes
    classes = ['Normal', 'Atypique']
    ax.set_xticks(range(len(classes)))
    ax.set_yticks(range(len(classes)))
    ax.set_xticklabels(classes)
    ax.set_yticklabels(classes)

    # Ajout d'une barre de couleur
    cbar = ax.figure.colorbar(heatmap, ax=ax)

    # Ajout des titres
    ax.set_xlabel('Prédictions')
    ax.set_ylabel('Vraies étiquettes')
    ax.set_title('Matrice de confusion')

    # Affichage de la figure
    plt.show()


Définition des hyperparamètres à essayer

In [ ]:
parameters = {
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'C': [1e-2, 1e-1, 1, 1e1],
    'gamma': ['scale', 'auto']
}

Dans notre cas où la classe négative représente environ 84% des échantillons, utiliser le rappel seul pourrait être trompeur. Le rappel mesure la capacité d'un modèle à identifier correctement les échantillons positifs parmi tous les échantillons positifs réels. 

En utilisant ``balanced_accuracy``, on donne une importance égale aux performances des deux classes. Cela permet de s'assurer que notre modèle ne se concentre pas uniquement sur la classe majoritaire (négatif), mais qu'il est également capable de prédire correctement la classe minoritaire (postive). 

Instanciations

In [ ]:
svmc = SVC()
grille = GridSearchCV(estimator=svmc, param_grid=parameters, scoring='balanced_accuracy', cv=2, verbose=3)

Exécuter la recherche de grille pour trouver la meilleure configuration de modèle en ajustant les modèles sur les données d'apprentissage et en évaluant leur performance à l'aide de la validation croisée

In [ ]:
resultats = grille.fit(X_train, y_train)

Affichage du meilleur modèle

In [ ]:
print('Le meilleur modèle :', resultats.best_params_)

In [ ]:
resultats.cv_results_ 

In [ ]:
svm = resultats.best_estimator_
y_true = y_test
y_pred = svm.predict(X_test)

In [ ]:
display_confusion_matrix(y_true, y_pred)

In [ ]:
accuracy_score(y_true,y_pred)

In [ ]:
print(classification_report(y_true,y_pred))

On regarde si on aurait obtenu de meilleurs résultats/un autre meilleur kernel avec d'autres fonctions de scoring pour le GridSearch

In [ ]:
def test_scoring_value(scoring):
    print('>>> Scoring => ', scoring)
    # determination du meilleur kernel pour la fonction de scoring
    svmc = SVC()
    grille = GridSearchCV(estimator=svmc, param_grid=parameters, scoring=scoring, cv=2)
    resultats = grille.fit(X_train, y_train)
    print('Le meilleur modèle :', resultats.best_params_)
    
    # confusion matrix & accuracy
    svm = resultats.best_estimator_
    y_true = y_test
    y_pred = svm.predict(X_test)
    print(confusion_matrix(y_true, y_pred))
    print(accuracy_score(y_true,y_pred))

In [ ]:
scorings = ['accuracy', 'balanced_accuracy', 'top_k_accuracy', 'average_precision', 'neg_brier_score', 'f1', 'neg_log_loss', 'precision', 'recall', 'jaccard', 'roc_auc']
for s in scorings:
    test_scoring_value(s)

### Recherche des meilleurs attributs au vu de leur impact sur l'erreur

Récupération des hyperparamètres optimaux

In [ ]:
try:
    C_opti = resultats.best_params_['C']
    kernel_opti = resultats.best_params_['kernel']
    gamma_opti = resultats.best_params_['gamma']
except:
    C_opti = 10
    kernel_opti = 'rbf'
    gamma_opti = 'auto'

In [ ]:
impact=[]
for i in range(1,10):
    print("####### NOMBRE DE COMPOSANTES PRINCIPALES ",i)
    X1=df.drop(df.columns[-1], axis=1)
    X=X1.iloc[:, :i].values
    labels=df.iloc[:, -1].replace([1.0,2.0,4.0], 0)
    labels=labels.replace(3.0, 1)
    Y=labels.values

    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3,random_state=42)
    
    svm=SVC(C=C_opti,kernel=kernel_opti, gamma=gamma_opti)
    svm.fit(X_train, y_train)
    y_pred=svm.predict(X_test)
    y_true = y_test
    print(classification_report(y_true,y_pred))
    impact.append({"Nombre de CP" :i,"accuracy":accuracy_score(y_true,y_pred)})

    #display_confusion_matrix(y_true, y_pred)

In [ ]:
x_values = [d['Nombre de CP'] for d in impact]
y_values = [d['accuracy'] for d in impact]

plt.plot(x_values, y_values, marker='o')
plt.xlabel('Nombre de CP')
plt.ylabel('accuracy')
plt.title('Evolution de la précision avec le nombre de CP')
plt.show()

## Représentation graphique

In [ ]:

scatter = plt.scatter(X[:, 0], X[:, 1], c=Y,edgecolors='k', cmap=plt.cm.coolwarm)
legend = plt.legend(*scatter.legend_elements(), title='Label')
# Afficher le graphique 2D
plt.xlabel('x1')
plt.ylabel('x2')
plt.title("SVM RBF All")
plt.show()


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Tracer les points en 3D avec une couleur basée sur la dimension supplémentaire
sc = ax.scatter(X[:, 0], X[:, 1],X[:, 2], c=Y, cmap=plt.cm.coolwarm,edgecolors='k')
legend = ax.legend(*sc.legend_elements(), title='Label')
ax.add_artist(legend)

# Ajouter des labels aux axes
ax.set_xlabel('Dim 1')
ax.set_ylabel('Dim 2')
ax.set_zlabel('Dim 3')

# Afficher le graphique 3D
plt.show()